Paso 0 — Reset seguro de la colección (opcional)

In [1]:
# --- Paso 0: reset opcional de la colección para evitar "ruido" previo ---
import chromadb

VDB_PATH = "./vectordb"
COLLECTION_NAME = "langchain"

def reset_collection():
    client = chromadb.PersistentClient(path=VDB_PATH)
    try:
        client.delete_collection(COLLECTION_NAME)
        print(f"🧹 Colección '{COLLECTION_NAME}' eliminada.")
    except Exception as e:
        print(f"(info) No se eliminó colección (quizá no existía): {e}")

# Si quieres limpiar todo antes de reindexar:
# reset_collection()


In [ ]:
# --- Paso 1: imports y utilidades ---
import os, re, pdfplumber
from typing import List, Tuple, Dict, Any
from collections import Counter

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document  # langchain_core.documents.Document en versiones nuevas

from dotenv import load_dotenv
import os

# Cargar variables del archivo .env
load_dotenv()

# Leer la API key desde el entorno
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


# Patrones de limpieza (encabezados/pies repetidos del PDF ejemplo)
CLEAN_PATTERNS = [
    r"FORMATO REQUISICIÓN.*",
    r"ECOPETROL DESARROLLO DE PROYECTO.*",
    r"Todos los derechos reservados.*",
    r"EDP-F-046.*Versión.*",
    r"MR.*#:.*CAS.*",
    r"Página\s+\d+\s+de\s+\d+",
    r"_{6,}",  # líneas de subrayado largas
]

def clean_text(text: str) -> str:
    for p in CLEAN_PATTERNS:
        text = re.sub(p, "", text, flags=re.IGNORECASE)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


# Palabras clave para asignar prioridad (ES/EN)
HD_HINTS = [
    r"\bAPI\s*675\b", r"HOJA\s*DE\s*DATOS", r"\bDATA\s*SHEET\b", r"\bDATASHEET\b",
    r"BOMB(A|S)?\s+(DOSIFICADORA|DOSING)"
]

ALCANCE_HINTS = [
    r"\bALCANCE\b", r"ALCANCE\s+DEL\s+SUMINISTRO", r"\bSCOPE\b", r"SCOPE\s+OF\s+SUPPLY"
]

REQ_HINTS = [
    # Español: requisitos explícitos
    r"\bREQUISITO(S)?\b",
    r"\bREQUERIMIENTO(S)?\b",
    r"REQUISITO(S)?\s+T(É|E)CNICO(S)?",
    r"CONDICIONES?\s+T(É|E)CNICAS?",
    r"CRITERIOS?\s+DE\s+DISE(Ñ|N)O",
    r"DATOS?\s+DE\s+DISE(Ñ|N)O",

    # Inglés: requisitos explícitos
    r"\bREQUIREMENT(S)?\b",
    r"DESIGN\s+REQUIREMENT(S)?",
    r"TECHNICAL\s+REQUIREMENT(S)?",
    r"\bSPECIFICATION(S)?\b",
    r"MINIMUM\s+REQUIREMENT(S)?",

    # Español: frases prescriptivas comunes
    r"\bEL\s+PROVEEDOR\s+DEBE\b",
    r"\bEL\s+VENDEDOR\s+DEBE\b",
    r"\bDEBERÁ\b",
    r"\bSE\s+REQUIERE(N)?\b",

    # Inglés: frases prescriptivas comunes
    r"\bSHALL\b",
    r"\bMUST\b",
    r"\bIS\s+REQUIRED\s+TO\b",
]


NORM_HINTS = [
    r"\bNORMAS?\b", r"\bEST[ÁA]NDA(RE|R)E?S?\b",
    r"\bC(Ó|O)DIGO(S)?\b", r"\bCODE(S)?\b", r"\bSTANDARD(S)?\b",
    r"\bSPEC(S|IFICATIONS)?\b", r"\bREGULATION(S)?\b",
    r"\bRETIE\b", r"\bAPI\b", r"\bASTM\b", r"\bASME\b", r"\bIEC\b", r"\bIEEE\b",
    r"APPLICABLE\s+(CODES|STANDARDS|SPECIFICATIONS)"
]


def matches_any(text: str, patterns: List[str]) -> bool:
    return any(re.search(p, text, flags=re.IGNORECASE) for p in patterns)

def detect_priority(text: str) -> str:
    if matches_any(text, HD_HINTS):      return "hoja_datos"
    if matches_any(text, ALCANCE_HINTS): return "alcance"
    if matches_any(text, REQ_HINTS):     return "requisito"
    if matches_any(text, NORM_HINTS):    return "norma"
    return "general"

def table_to_text_and_kv(rows: List[List[str]]) -> Tuple[str, Dict[str, str] | None]:
    """Convierte una tabla (lista de filas) a texto legible y, si parece key-value (2 columnas), devuelve un dict kv."""
    clean_rows = []
    for r in rows:
        if not r:
            continue
        rr = [(c or "").strip() for c in r]
        if any(rr):
            clean_rows.append(rr)

    kv = None
    if clean_rows and max(len(r) for r in clean_rows) <= 2 and len(clean_rows) >= 2:
        kv = {}
        for r in clean_rows:
            if len(r) >= 2 and r[0] and r[1]:
                kv[r[0]] = r[1]

    text = "\n".join(" | ".join(r) for r in clean_rows)
    return text, kv if kv else None


def kv_from_text_page(text: str) -> Dict[str, str]:
    """Extrae pares clave-valor típicos de una Hoja de Datos por regex"""
    t = " ".join(text.split()).upper()
    kv = {}

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*GPH", t)
    if m: kv["caudal_gph"] = m.group(1)

    m = re.search(r"@\s*(\d+(?:[.,]\d+)?)\s*PSI", t)
    if m: kv["presion_psig"] = m.group(1)

    m = re.search(r"(\d+)\s*-\s*(\d+)\s*CPS", t)
    if m:
        kv["rango_cps_min"] = m.group(1)
        kv["rango_cps_max"] = m.group(2)

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*HP", t)
    if m: kv["hp_motor"] = m.group(1)

    m = re.search(r"\b(\d+)\s*\+\s*(\d+)\b", t)
    if m:
        kv["bombas_operativas"] = m.group(1)
        kv["bombas_respaldo"] = m.group(2)

    if "304" in t: kv["material_tanque"] = "304SS"
    if "316" in t: kv["material_mojado"] = "316SS"
    if "API 675" in t: kv["api_675"] = "true"

    return kv


def kv_from_table_cells(table_rows: List[List[str]]) -> Dict[str, str]:
    """Convierte tabla en texto y aplica regex de kv_from_text_page"""
    lines = []
    for row in table_rows:
        cells = [c.strip() for c in row if c]
        if any(cells):
            lines.append(" | ".join(cells))
    table_text = "\n".join(lines)
    return kv_from_text_page(table_text)



In [3]:
# --- Paso 2: extracción de contenido del PDF ---
import json
from typing import List

def kv_from_text_page(text: str) -> dict:
    """
    Extrae pares clave-valor típicos de una Hoja de Datos desde texto plano (ES/EN).
    Usa regex para capturar caudal, presión, HP, materiales, etc.
    """
    t = " ".join(text.split()).upper()
    kv = {}

    # Caudal (GPH o LPH)
    m = re.search(r"(\d+(?:[.,]\d+)?)\s*(GPH|GALLON[S]?\s+PER\s+HOUR)", t)
    if m: kv["caudal_gph"] = m.group(1)

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*(LPH|LITROS?\s+POR\s+HORA)", t)
    if m: kv["caudal_lph"] = m.group(1)

    # Presión (PSI o BAR)
    m = re.search(r"(\d+(?:[.,]\d+)?)\s*PSI", t)
    if m: kv["presion_psig"] = m.group(1)

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*BAR", t)
    if m: kv["presion_bar"] = m.group(1)

    # Potencia (HP o KW)
    m = re.search(r"(\d+(?:[.,]\d+)?)\s*HP", t)
    if m: kv["hp_motor"] = m.group(1)

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*KW", t)
    if m: kv["kw_motor"] = m.group(1)

    # Viscosidad (cps)
    m = re.search(r"(\d+)\s*-\s*(\d+)\s*CPS", t)
    if m:
        kv["rango_cps_min"] = m.group(1)
        kv["rango_cps_max"] = m.group(2)

    # Bombas operativas + respaldo (ej. "1+1")
    m = re.search(r"\b(\d+)\s*\+\s*(\d+)\b", t)
    if m:
        kv["bombas_operativas"] = m.group(1)
        kv["bombas_respaldo"] = m.group(2)

    # Materiales comunes
    if "304" in t: kv["material_tanque"] = "304SS"
    if "316" in t: kv["material_mojado"] = "316SS"

    # API 675
    if "API 675" in t: kv["api_675"] = "true"

    return kv


def extract_pdf_content(filepath: str) -> List[Document]:
    """
    Extrae texto y tablas del PDF con metadatos enriquecidos:
    - tipo (text/table)
    - prioridad (hoja_datos, alcance, requisito, norma, general)
    - kv cuando se detecta info técnica
    """
    docs: List[Document] = []

    with pdfplumber.open(filepath) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            # --- Texto plano ---
            txt = page.extract_text() or ""
            clean = clean_text(txt)

            if clean:
                prio = detect_priority(clean)
                md = {"source": filepath, "page": i, "type": "text", "prio": prio}

                kv = kv_from_text_page(clean)
                if kv:
                    md["kv"] = json.dumps(kv, ensure_ascii=False)
                docs.append(Document(page_content=clean, metadata=md))

            # --- Tablas ---
            try:
                tables = page.extract_tables() or []
            except Exception:
                tables = []

            for trows in tables:
                table_text, kv_table = table_to_text_and_kv(trows)
                table_text = clean_text(table_text)
                if not table_text:
                    continue

                prio = detect_priority(table_text)
                md = {"source": filepath, "page": i, "type": "table", "prio": prio}

                kv_detected = kv_from_text_page(table_text)  # también extraemos de texto de tabla
                if kv_table:
                    kv_detected = {**kv_detected, **kv_table}

                if kv_detected:
                    md["kv"] = json.dumps(kv_detected, ensure_ascii=False)

                docs.append(Document(page_content=table_text, metadata=md))

    return docs


In [4]:
# ============================================
# Paso 3 - Extractor de PDF (texto + tablas + HD + prioridades)
# ============================================
import pdfplumber, json
from langchain.schema import Document
from typing import List

def extract_pdf_content(filepath: str) -> List[Document]:
    docs: List[Document] = []
    
    with pdfplumber.open(filepath) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            # --- 1) Texto plano ---
            raw_text = page.extract_text() or ""
            clean = clean_text(raw_text)
            if clean:
                md = {
                    "source": filepath,
                    "page": i,
                    "type": "text",
                    "prio": detect_priority(clean)  # prioridad automática
                }

                # Regex: intentar extraer posibles valores técnicos (caudal, presión, hp…)
                kv_text = kv_from_text_page(clean)
                if kv_text:
                    md["kv"] = json.dumps(kv_text, ensure_ascii=False)

                docs.append(Document(page_content=clean, metadata=md))

            # --- 2) Tablas ---
            try:
                tables = page.extract_tables()
            except Exception:
                tables = []

            for t in tables:
                if not t:
                    continue

                table_text = "\n".join(
                    [" | ".join(filter(None, row)) for row in t if any(row)]
                )
                table_text = clean_text(table_text)

                md = {
                    "source": filepath,
                    "page": i,
                    "type": "table",
                    "prio": detect_priority(table_text)
                }

                # Extraer key-values si la tabla tiene estructura tipo HD
                kv_table = kv_from_table_cells(t)
                if kv_table:
                    md["kv"] = json.dumps(kv_table, ensure_ascii=False)

                if table_text:
                    docs.append(Document(page_content=table_text, metadata=md))

    return docs


In [5]:
# ============================================
# Paso 4 - Indexar en Chroma (persistente)
# ============================================
from collections import Counter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Configuración global
VDB_PATH = "./vectordb"
COLLECTION_NAME = "autoselx_docs"

def reset_collection():
    """Elimina la colección persistente (para recargar desde cero)."""
    import shutil, os
    if os.path.exists(VDB_PATH):
        shutil.rmtree(VDB_PATH)
        print(f"🗑️ Colección eliminada en {VDB_PATH}")

def add_files_to_vectordb(filepath: str, reset: bool = False):
    if reset:
        reset_collection()

    # --- 1) Extraer documentos brutos (texto + tablas + HD) ---
    raw_docs = extract_pdf_content(filepath)

    # --- 2) Chunking con preservación de prio + kv ---
    splits = chunk_documents(raw_docs)

    # --- 3) Indexar en Chroma persistente ---
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=OpenAIEmbeddings(),
        persist_directory=VDB_PATH,
        collection_name=COLLECTION_NAME
    )

    # --- 4) Diagnóstico de indexación ---
    by_type = Counter(d.metadata.get("type", "text") for d in splits)
    by_prio = Counter(d.metadata.get("prio", "general") for d in splits)
    total_kv = sum(1 for d in splits if "kv" in d.metadata)

    print(f"✅ {len(splits)} fragmentos indexados en {VDB_PATH} (colección '{COLLECTION_NAME}')")
    print("   📊 Por tipo:", dict(by_type))
    print("   📊 Por prioridad:", dict(by_prio))
    print(f"   🔑 Documentos con KV extraídos: {total_kv}")

    return vectorstore


In [6]:
# ============================================
# Paso 5 - Utilidad para inspeccionar fuentes en la colección
# ============================================
from collections import Counter
import chromadb

def get_unique_sources_list():
    """Devuelve lista de fuentes únicas indexadas y estadísticas de metadatos."""
    client = chromadb.PersistentClient(path=VDB_PATH)
    collection = client.get_collection(COLLECTION_NAME)

    data = collection.get(include=["documents", "metadatas"])
    metadatas = data.get("metadatas", [])

    sources = []
    for m in metadatas:
        if not m:
            continue
        src = m.get("source")
        if src:
            sources.append(src.split("/")[-1])

    # 📊 Diagnóstico
    by_prio = Counter(m.get("prio", "general") for m in metadatas if m)
    by_type = Counter(m.get("type", "text") for m in metadatas if m)

    print(f"📁 Fuentes únicas en la colección ({COLLECTION_NAME}): {len(set(sources))}")
    print("   ", sorted(set(sources)))
    print("   📊 Distribución por prioridad:", dict(by_prio))
    print("   📊 Distribución por tipo:", dict(by_type))

    return sorted(set(sources))

# Ejemplo:
# fuentes = get_unique_sources_list()
# print("Fuentes disponibles:", fuentes)




In [7]:
# ============================================
# Paso 5.1 - Chunking con preservación de metadatos
# ============================================
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

def chunk_documents(docs: List[Document], chunk_size: int = 800, chunk_overlap: int = 100) -> List[Document]:
    """
    Divide documentos largos en chunks manejables para embeddings,
    conservando metadatos (source, page, type, prio, kv).
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )

    chunks: List[Document] = []

    for doc in docs:
        splits = splitter.split_text(doc.page_content)
        for idx, chunk in enumerate(splits, start=1):
            new_meta = doc.metadata.copy()
            new_meta["chunk_id"] = f"{new_meta.get('page','na')}_{idx}"

            # Mantener prio y kv
            if "prio" not in new_meta:
                new_meta["prio"] = "general"
            if "kv" in doc.metadata:
                new_meta["kv"] = doc.metadata["kv"]

            chunks.append(Document(page_content=chunk, metadata=new_meta))

    return chunks


In [8]:
# --- Paso 6: correr el pipeline ---
if __name__ == "__main__":
    # Limpia todo si quieres partir desde cero:
    # reset_collection()

    vectorstore = add_files_to_vectordb(
        "./data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf",
        reset=False  # pon True si quieres borrar la colección antes
    )


✅ 613 fragmentos indexados en ./vectordb (colección 'autoselx_docs')
   📊 Por tipo: {'text': 386, 'table': 227}
   📊 Por prioridad: {'alcance': 108, 'norma': 80, 'general': 125, 'hoja_datos': 197, 'requisito': 103}
   🔑 Documentos con KV extraídos: 151


In [9]:
# ============================================
# Paso 7 - Debug: explorar fragmentos
# ============================================
import json

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
query = "¿Cuál es el alcance del suministro?"
results = retriever.get_relevant_documents(query)

print(f"🔎 Consulta: {query}")
print(f"📊 Fragmentos recuperados: {len(results)}")

for idx, r in enumerate(results, start=1):
    print(f"\n--- CHUNK {idx} ---")
    print(r.page_content[:500], "...")
    print("Metadatos:", r.metadata)
    if "kv" in r.metadata:
        try:
            kv_dict = json.loads(r.metadata["kv"])
            print("KV dict:", json.dumps(kv_dict, indent=2, ensure_ascii=False))
        except Exception:
            print("KV (raw):", r.metadata["kv"])



/tmp/ipykernel_34383/2361241895.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


🔎 Consulta: ¿Cuál es el alcance del suministro?
📊 Fragmentos recuperados: 5

--- CHUNK 1 ---
SECCIÓN | DESCRIPCIÓN
2 | ALCANCE DEL SUMINISTRO
10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN) ...
Metadatos: {'type': 'table', 'chunk_id': '1_1', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'alcance', 'page': 1}

--- CHUNK 2 ---
SECCIÓN | DESCRIPCIÓN
2 | ALCANCE DEL SUMINISTRO
10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN) ...
Metadatos: {'chunk_id': '1_1', 'page': 1, 'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'alcance'}

--- CHUNK 3 ---
SECCIÓN | DESCRIPCIÓN
2 | ALCANCE DEL SUMINISTRO
10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN) ...
Metadatos: {'page': 1, 'chunk_id': '1_1', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'table', 'prio': 'alcance'}

--- CHUNK 4 ---
SECCIÓN | DESCRIPCIÓN
2 | ALCANCE DEL SUMINIST

In [10]:
# ==========================
# Complemento Paso 7 - Función auxiliar de debug
# ==========================
def debug_query(vectorstore, query, prio=None, k=5):
    """
    Ejecuta una búsqueda con filtro de metadatos (cuando prio != None).
    Hace fallback si no hay resultados en la prioridad especificada.
    """
    import json

    # Mapeo de "prios aceptables" por intención (fallbacks):
    prio_fallbacks = {
        "alcance":   ["alcance", "general"],
        "requisito": ["requisito", "hoja_datos", "general"],
        "norma":     ["norma", "general"],
        "hoja_datos":["hoja_datos", "requisito", "general"]
    }

    print(f"\n🔎 DEBUG QUERY: '{query}' (prio={prio})\n")

    def run_search(allowed_prios, topk):
        if allowed_prios is None:
            # sin filtro
            return vectorstore.similarity_search(query, k=topk)
        else:
            # filtro por metadatos (Chroma soporta {'$in': [...]})
            return vectorstore.similarity_search(
                query, k=topk, filter={"prio": {"$in": allowed_prios}}
            )

    # 1) Intento con filtro (si prio se pidió)
    if prio:
        allowed = prio_fallbacks.get(prio, [prio])
        docs = run_search(allowed_prios=allowed, topk=max(k, 10))
        if not docs:
            # 2) Fallback agresivo: ampliar K y permitir 'general' siempre
            allowed_wide = list(set(allowed + ["general"]))
            docs = run_search(allowed_prios=allowed_wide, topk=50)
            if not docs:
                print("⚠️ No se encontraron resultados con esa prioridad (ni con fallback).")
                return
    else:
        docs = run_search(allowed_prios=None, topk=k)

    print(f"📊 Fragmentos recuperados: {len(docs)}\n")
    for i, r in enumerate(docs, 1):
        print(f"--- CHUNK {i} ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)
        if "kv" in r.metadata:
            try:
                print("KV dict:", json.loads(r.metadata["kv"]))
            except Exception:
                print("KV (raw):", r.metadata["kv"])
        print()


In [11]:
# ==========================
# Pruebas de debug por prioridad
# ==========================

# Alcance del suministro
debug_query(vectorstore, "alcance del suministro", prio="alcance")


🔎 DEBUG QUERY: 'alcance del suministro' (prio=alcance)

📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO 10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'alcance', 'type': 'table', 'chunk_id': '1_1', 'page': 1}

--- CHUNK 2 ---
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO 10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)
Metadatos: {'prio': 'alcance', 'page': 1, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'table', 'chunk_id': '1_1'}

--- CHUNK 3 ---
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO 10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)
Metadatos: {'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '1_1', 'prio': 'alcance', 'page': 1}

--- CHUNK 4 ---
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO

In [12]:
debug_query(vectorstore, "API 675 caudal presión material", prio="hoja_datos")


🔎 DEBUG QUERY: 'API 675 caudal presión material' (prio=hoja_datos)

📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
UNIDAD COMPLETA      CERTIFICADOS DE MATERIAL APLICADO A UNIDAD PAQUETE OTROS: API 675 BOMBAS DE DESPLAZAMIENTO POSITIVO - VOL. CONTROLADO  OTROS: | | | | | | | | | | | | | | | | | | | |   | PREPARACIÓN DE ENVÍO | | | | | | | | | | | | | | | | | | | |   | PREPARACIÓN DOMÉSTICA:  POR PROVEEDOR  POR COMPRADOR  OBSERVACIONES CAJA DE EXPORTACIÓN:  POR PROVEEDOR  POR COMPRADOR  OBSERVACIONE
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'hoja_datos', 'type': 'table', 'chunk_id': '42_11', 'page': 42, 'kv': '{"caudal_gph": "12-18", "presion_psig": "40", "hp_motor": "1", "kw_motor": "5", "rango_cps_min": "500", "rango_cps_max": "1000", "bombas_operativas": "1", "bombas_respaldo": "1", "material_tanque": "304SS", "material_mojado": "316SS", "api_675": "true"}'}
KV dict: {'caudal_gph': '12-18', 'presion_psig': '40', 'hp

In [13]:
debug_query(vectorstore, "RETIE y normas aplicables", prio="norma")


🔎 DEBUG QUERY: 'RETIE y normas aplicables' (prio=norma)

📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
puede ser hecha sin permiso escrito. Ningún párrafo de esta publicación puede ser reproducido, copiado o transmitido digitalmente sin un consentimiento escrito o de acuerdo con las leyes que regulan los derechos de autor y con base en la regulación vigente.
Metadatos: {'prio': 'norma', 'type': 'text', 'page': 4, 'chunk_id': '4_4', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK 2 ---
puede ser hecha sin permiso escrito. Ningún párrafo de esta publicación puede ser reproducido, copiado o transmitido digitalmente sin un consentimiento escrito o de acuerdo con las leyes que regulan los derechos de autor y con base en la regulación vigente.
Metadatos: {'page': 14, 'type': 'text', 'prio': 'norma', 'chunk_id': '14_5', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK 3 ---
puede ser hecha sin permiso escrito.

In [14]:
debug_query(vectorstore, "requisitos técnicos del sistema de dosificación", prio="requisito")


🔎 DEBUG QUERY: 'requisitos técnicos del sistema de dosificación' (prio=requisito)

📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
de acuerdo con la hoja de datos Paquete De Química STAP EC3 CAS-09991-MER-HD-000003 a fin de garantizar una unidad funcional completa. El PROVEEDOR debe cumplir con los requerimientos mínimos requeridos por el cliente para entregar un sistema de dosificación efectivo y que cumpla con las expectativas en la operación y producción de la estación. EDP-F-046 – 20/11/2017 V-1 8/36 Documento Rector: EDP
Metadatos: {'page': 8, 'prio': 'hoja_datos', 'chunk_id': '8_5', 'type': 'text', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK 2 ---
de acuerdo con la hoja de datos Paquete De Química STAP EC3 CAS-09991-MER-HD-000003 a fin de garantizar una unidad funcional completa. El PROVEEDOR debe cumplir con los requerimientos mínimos requeridos por el cliente para entregar un sistema de dosificación efectivo y que cumpla con las expe

In [15]:
# ============================================
# Paso 8 - Probar extracción de Hoja de Datos API 675
# ============================================
import json

def buscar_hojas_datos(pregunta: str = "hoja de datos bomba", k: int = 12):
    # --- 1) Buscamos primero chunks con prio=hoja_datos ---
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    results = retriever.get_relevant_documents(pregunta)
    hd_chunks = [r for r in results if r.metadata.get("prio") == "hoja_datos"]

    # --- 2) Si hay pocos resultados, ampliamos con los que tengan KV ---
    if len(hd_chunks) < 3:
        kv_chunks = [r for r in results if "kv" in r.metadata]
        filtrados = hd_chunks + kv_chunks
    else:
        filtrados = hd_chunks

    print(f"🔎 Total de chunks candidatos (HD/kv): {len(filtrados)}")

    count = 0
    for r in filtrados:
        has_kv = "kv" in r.metadata
        if has_kv:
            count += 1

        flag = "🔑" if has_kv else ""
        print(f"\n--- CHUNK HD {flag} ---")
        print(f"p.{r.metadata.get('page')} | {r.metadata.get('type')} | prio={r.metadata.get('prio')}")
        print(r.page_content[:400])

        if has_kv:
            try:
                kv = json.loads(r.metadata["kv"])
                print("KV:", kv)
            except Exception:
                print("KV (raw):", r.metadata["kv"])

    print(f"✅ Chunks con KV reales: {count}")
    return filtrados


# Ejemplo:
buscar_hojas_datos("Datos de diseño de bomba dosificadora API 675 (caudal, presión, HP)")



🔎 Total de chunks candidatos (HD/kv): 12

--- CHUNK HD 🔑 ---
p.43 | text | prio=hoja_datos
96 2 agua, para la introducción del polímero sólido, agitacion, alarma para vaciado y llenado. para la dosificacion debe contar con Una (1) bomba dosificadora de diafragma para la inyección de polímero líquido preparado de 1Hp,12 - 18 gph @ 100psi, API 675, con una (1) de respaldo accionadas por motores eléctricos. El
paquete contara con un sistema de control PLC dedicado a gestionar todas las fun
KV: {'caudal_gph': '18', 'presion_psig': '100', 'hp_motor': '1', 'material_tanque': '304SS', 'api_675': 'true'}

--- CHUNK HD 🔑 ---
p.43 | text | prio=hoja_datos
96 2 agua, para la introducción del polímero sólido, agitacion, alarma para vaciado y llenado. para la dosificacion debe contar con Una (1) bomba dosificadora de diafragma para la inyección de polímero líquido preparado de 1Hp,12 - 18 gph @ 100psi, API 675, con una (1) de respaldo accionadas por motores eléctricos. El
paquete contara con un sis

[Document(metadata={'kv': '{"caudal_gph": "18", "presion_psig": "100", "hp_motor": "1", "material_tanque": "304SS", "api_675": "true"}', 'type': 'text', 'chunk_id': '43_3', 'prio': 'hoja_datos', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'page': 43}, page_content='96 2 agua, para la introducción del polímero sólido, agitacion, alarma para vaciado y llenado. para la dosificacion debe contar con Una (1) bomba dosificadora de diafragma para la inyección de polímero líquido preparado de 1Hp,12 - 18 gph @ 100psi, API 675, con una (1) de respaldo accionadas por motores eléctricos. El\npaquete contara con un sistema de control PLC dedicado a gestionar todas las funciones del paquete, sus cajas (Potencia, Instrumentación, Instrumentos y Control NEMA 4X (Área no clasificada). La instrumentación mínima asociada al paquete está indicada en el P&ID CAS-09991-PRO-PID-000012 - STAP EC3.\n97'),
 Document(metadata={'type': 'text', 'prio': 'hoja_datos', 'source': './

In [16]:
# ============================================
# Paso 8.1 - Consolidar Hoja de Datos API 675
# ============================================
import json

def extraer_campos_extras(texto: str) -> Dict[str, str]:
    """Regex para campos no capturados aún en kv_from_text_page"""
    t = texto.upper()
    kv = {}

    # Fluido de operación
    m = re.search(r"FLUIDO\s+DE\s+OPERACI[ÓO]N\s+SI\s+([A-ZÁÉÍÓÚ0-9\s]+)", t)
    if m:
        kv["fluido"] = m.group(1).title()

    # Viscosidad cP
    m = re.search(r"VISCO(SIDAD)?[^0-9]*(\d{2,5})\s*-\s*(\d{2,5})\s*CP", t)
    if m:
        kv["viscosidad_cps"] = f"{m.group(2)}-{m.group(3)}"

    # Eficiencia %
    m = re.search(r"EFICIENCIA.*?(\d{1,3})\s*%|≥\s*(\d{1,3})", t)
    if m:
        kv["eficiencia_pct"] = m.group(1) or m.group(2)

    # Clasificación de área
    m = re.search(r"CLASIFICACI[ÓO]N\s+DEL\s+ÁREA\s+NO\s+([A-Z\s]+)", t)
    if m:
        kv["clasificacion_area"] = m.group(1).title()

    return kv

def consolidar_hoja_datos(pregunta="hoja de datos bomba API 675", k=12) -> Dict[str, Any]:
    docs = buscar_hojas_datos(pregunta, k=k)
    consolidado = {}

    for d in docs:
        # 1) KV en metadata
        if "kv" in d.metadata:
            try:
                kv = json.loads(d.metadata["kv"])
                consolidado.update(kv)
            except Exception:
                pass

        # 2) Extra campos con regex sobre el contenido
        extras = extraer_campos_extras(d.page_content)
        consolidado.update(extras)

    print("\n✅ JSON consolidado:")
    print(json.dumps(consolidado, indent=2, ensure_ascii=False))
    return consolidado

# Ejemplo:
consolidar_hoja_datos("Datos de diseño de bomba dosificadora API 675")


🔎 Total de chunks candidatos (HD/kv): 12

--- CHUNK HD 🔑 ---
p.43 | text | prio=hoja_datos
96 2 agua, para la introducción del polímero sólido, agitacion, alarma para vaciado y llenado. para la dosificacion debe contar con Una (1) bomba dosificadora de diafragma para la inyección de polímero líquido preparado de 1Hp,12 - 18 gph @ 100psi, API 675, con una (1) de respaldo accionadas por motores eléctricos. El
paquete contara con un sistema de control PLC dedicado a gestionar todas las fun
KV: {'caudal_gph': '18', 'presion_psig': '100', 'hp_motor': '1', 'material_tanque': '304SS', 'api_675': 'true'}

--- CHUNK HD 🔑 ---
p.43 | text | prio=hoja_datos
96 2 agua, para la introducción del polímero sólido, agitacion, alarma para vaciado y llenado. para la dosificacion debe contar con Una (1) bomba dosificadora de diafragma para la inyección de polímero líquido preparado de 1Hp,12 - 18 gph @ 100psi, API 675, con una (1) de respaldo accionadas por motores eléctricos. El
paquete contara con un sis

{'caudal_gph': '12-18',
 'presion_psig': '100',
 'hp_motor': '1',
 'material_tanque': '304SS',
 'api_675': 'true'}

In [17]:
# ============================================
# Paso 9 - Inspeccionar metadatos del Alcance
# ============================================

def buscar_alcance(pregunta: str = "alcance del suministro", k: int = 8):
    """Busca chunks relacionados con alcance (prio=alcance) y muestra metadatos"""
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k, "filter": {"prio": "alcance"}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Total de chunks recuperados (prio=alcance): {len(results)}")
    for r in results:
        print(
            f"\n--- CHUNK p.{r.metadata.get('page')} | "
            f"{r.metadata.get('type')} | prio={r.metadata.get('prio')} ---"
        )
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

    return results

# Ejemplo de prueba:
buscar_alcance()



🔎 Total de chunks recuperados (prio=alcance): 8

--- CHUNK p.1 | table | prio=alcance ---
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO 10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)
Metadatos: {'page': 1, 'chunk_id': '1_1', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'alcance', 'type': 'table'}

--- CHUNK p.1 | table | prio=alcance ---
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO 10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)
Metadatos: {'chunk_id': '1_1', 'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'page': 1, 'prio': 'alcance'}

--- CHUNK p.1 | table | prio=alcance ---
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO 10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '1_1', 'type': 'table', 'prio': 'alcance', 'page': 1}

--- CHUNK p.1 | table | pri

[Document(metadata={'page': 1, 'chunk_id': '1_1', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'alcance', 'type': 'table'}, page_content='SECCIÓN | DESCRIPCIÓN\n2 | ALCANCE DEL SUMINISTRO\n10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)'),
 Document(metadata={'chunk_id': '1_1', 'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'page': 1, 'prio': 'alcance'}, page_content='SECCIÓN | DESCRIPCIÓN\n2 | ALCANCE DEL SUMINISTRO\n10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)'),
 Document(metadata={'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '1_1', 'type': 'table', 'prio': 'alcance', 'page': 1}, page_content='SECCIÓN | DESCRIPCIÓN\n2 | ALCANCE DEL SUMINISTRO\n10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)'),
 Document(metadata={'page': 1, 'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_2003

In [18]:
# ============================================
# Paso 10 - Validar TODOS los metadatos (corregido)
# ============================================
from collections import Counter

def validar_metadatos_globales():
    """Valida consistencia de metadatos en la colección completa."""
    collection = vectorstore._collection
    data = collection.get(include=["metadatas", "documents"])

    total_docs = len(data["documents"])
    print(f"🔎 Total de registros en la colección: {total_docs}")

    errores = 0
    prio_counts = Counter()
    type_counts = Counter()
    kv_count = 0
    seen_ids = set()
    duplicados = 0

    for i, meta in enumerate(data["metadatas"]):
        doc_id = data["ids"][i]  # aquí sí podemos usarlo, ya viene siempre

        # Detectar duplicados de ID
        if doc_id in seen_ids:
            duplicados += 1
        else:
            seen_ids.add(doc_id)

        # Validar claves mínimas
        if not all(key in meta for key in ["source", "page", "type"]):
            print(f"⚠️ Registro {i} con metadatos incompletos:", meta)
            errores += 1
            continue

        # Validar valores
        if not isinstance(meta.get("page"), int) or meta["page"] <= 0:
            print(f"⚠️ Registro {i} con página inválida:", meta)
            errores += 1

        if not meta.get("type"):
            print(f"⚠️ Registro {i} con tipo vacío:", meta)
            errores += 1

        # Contadores útiles
        prio_counts[meta.get("prio", "none")] += 1
        type_counts[meta.get("type", "unknown")] += 1
        if "kv" in meta:
            kv_count += 1

    # Resumen
    if errores == 0:
        print("✅ Todos los documentos tienen metadatos completos y válidos.")
    else:
        print(f"⚠️ Se encontraron {errores} problemas de metadatos.")

    if duplicados > 0:
        print(f"⚠️ Se detectaron {duplicados} duplicados de IDs.")

    print("\n📊 Distribución por prioridad:", dict(prio_counts))
    print("📊 Distribución por tipo:", dict(type_counts))
    print(f"📊 Total de documentos con KV (tablas clave-valor): {kv_count}")

# Ejemplo:
validar_metadatos_globales()



🔎 Total de registros en la colección: 7369
✅ Todos los documentos tienen metadatos completos y válidos.

📊 Distribución por prioridad: {'alcance': 1298, 'general': 1604, 'hoja_datos': 2370, 'norma': 988, 'requisito': 1109}
📊 Distribución por tipo: {'text': 4632, 'table': 2737}
📊 Total de documentos con KV (tablas clave-valor): 1836


In [19]:
# ============================================
# Paso 11 - Buscar solo tablas o solo texto
# ============================================
def buscar_por_tipo(tipo: str = "table", pregunta: str = "bomba dosificadora", k: int = 5):
    """
    Recupera fragmentos filtrando por tipo ('table' o 'text').
    """
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k, "filter": {"type": tipo}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"\n🔎 Resultados filtrados por tipo='{tipo}' | pregunta='{pregunta}' → {len(results)} encontrados")
    for i, r in enumerate(results, 1):
        print(f"\n--- CHUNK {i} ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

        if "kv" in r.metadata:  # Mostrar KV si existe
            try:
                import json
                print("KV:", json.loads(r.metadata["kv"]))
            except Exception:
                print("KV (raw):", r.metadata["kv"])

    return results


# Ejemplos de prueba:
buscar_por_tipo("table", "tags y cantidades")
buscar_por_tipo("text", "alcance del suministro")




🔎 Resultados filtrados por tipo='table' | pregunta='tags y cantidades' → 5 encontrados

--- CHUNK 1 ---
| | requerían clasificación.
Metadatos: {'chunk_id': '51_1', 'page': 51, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'table', 'prio': 'general'}

--- CHUNK 2 ---
requerían clasificación.
Metadatos: {'prio': 'general', 'page': 51, 'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '51_1'}

--- CHUNK 3 ---
requerían clasificación.
Metadatos: {'type': 'table', 'page': 51, 'chunk_id': '51_1', 'prio': 'general', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK 4 ---
requerían clasificación.
Metadatos: {'type': 'table', 'chunk_id': '51_1', 'prio': 'general', 'page': 51, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK 5 ---
requerían clasificación.
Metadatos: {'prio': 'general', 'page': 51, 'type': 'table', 's

[Document(metadata={'page': 58, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'text', 'chunk_id': '58_2', 'prio': 'hoja_datos'}, page_content='2.0 ALCANCE\nEste documento describe el alcance del suministro y servicios del PROVEEDOR, sus\nresponsabilidades, plan de ejecución y demás aspectos generales para el diseño; fabricación;\nsuministro; asistencia para el montaje y pruebas del PAQUETE de inyección de químicos de Polímero\nAniónico AX-75317 y los Quill´s de Inyección Retráctil roscado, completamente operacionales y\nfuncionales.\nLos detalles relevantes a los requerimientos de proceso, características constructivas, garantías de\ndesempeño y otros requerimientos están incluidos en este documento.\nEsta especificación debe ser leída en conjunto con:\n• CAS-09991-MER-HD-000003: HOJA DE DATOS PAQUETE DE QUÍMICA STAP EC3.\n• CAS-09991-PRO-PID-000012: DIAGRAMA DE TUBERÍA E INSTRUMENTACIÓN SISTEMA'),
 Document(metadata={'type': 'text', 'prio': 'ho

In [20]:
# ============================================
# Paso 11.1 - Comparar resultados table vs text
# ============================================
from collections import Counter
import json

def comparar_table_vs_text(pregunta="alcance del suministro", k=6):
    resultados = {}
    for tipo in ["table", "text"]:
        retriever = vectorstore.as_retriever(
            search_kwargs={"k": k, "filter": {"type": tipo}}
        )
        docs = retriever.get_relevant_documents(pregunta)
        resultados[tipo] = docs

        print(f"\n🔎 Resultados filtrados por tipo='{tipo}' → {len(docs)} encontrados")
        prio_counter = Counter([d.metadata.get("prio", "none") for d in docs])
        kv_count = sum(1 for d in docs if "kv" in d.metadata)

        print(f"📊 Distribución de prioridades: {dict(prio_counter)}")
        print(f"📊 Con KV extraídos: {kv_count}")

        for r in docs:
            print("\n--- CHUNK ---")
            print(r.page_content[:300].replace("\n", " "))
            print("Metadatos:", r.metadata)
            if "kv" in r.metadata:
                try:
                    print("KV:", json.loads(r.metadata["kv"]))
                except:
                    print("KV (raw):", r.metadata["kv"])

    # Comparación final
    print("\n================= RESUMEN COMPARATIVO =================")
    for tipo in ["table", "text"]:
        print(f"Tipo: {tipo} | Total: {len(resultados[tipo])}")
        kv_total = sum(1 for d in resultados[tipo] if "kv" in d.metadata)
        print(f" → Con KV: {kv_total}")
        prio_counter = Counter([d.metadata.get("prio", "none") for d in resultados[tipo]])
        print(f" → Prioridades: {dict(prio_counter)}")

# Ejemplo de prueba
comparar_table_vs_text("tags y cantidades", k=5)
comparar_table_vs_text("alcance del suministro", k=5)



🔎 Resultados filtrados por tipo='table' → 5 encontrados
📊 Distribución de prioridades: {'general': 5}
📊 Con KV extraídos: 0

--- CHUNK ---
| | requerían clasificación.
Metadatos: {'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'general', 'page': 51, 'chunk_id': '51_1'}

--- CHUNK ---
requerían clasificación.
Metadatos: {'prio': 'general', 'chunk_id': '51_1', 'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'page': 51}

--- CHUNK ---
requerían clasificación.
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '51_1', 'page': 51, 'prio': 'general', 'type': 'table'}

--- CHUNK ---
requerían clasificación.
Metadatos: {'chunk_id': '51_1', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'general', 'page': 51, 'type': 'table'}

--- CHUNK ---
requerían clasificación.
Metadatos: {'prio': 'general', 'pag

In [21]:
# ============================================
# Paso 12 - Buscar en una página específica
# ============================================
def buscar_por_pagina(pagina: int, pregunta: str):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 5, "filter": {"page": pagina}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Resultados en página {pagina}: {len(results)}")
    for r in results:
        print("\n--- CHUNK ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

# Ejemplo:
buscar_por_pagina(32, "alcance del suministro")
buscar_por_pagina(42, "caudal de la bomba")


🔎 Resultados en página 32: 5

--- CHUNK ---
componentes, o dieciocho (18) meses a partir de la entrega a satisfacción de estos por parte de ECOPETROL S.A. Durante este período la garantía debe cubrir la reparación o reemplazo de los componentes defectuosos por parte del VENDEDOR incluyendo todas las labores, equipos, transporte y demás costos generados y requeridos para llevar a cabo dicha reparación y/o reemplazo. 10. ANEXOS 10.1. LISTA DE
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'page': 32, 'type': 'text', 'prio': 'hoja_datos', 'kv': '{"caudal_gph": "18", "presion_psig": "100", "hp_motor": "1", "material_tanque": "304SS", "api_675": "true"}', 'chunk_id': '32_3'}

--- CHUNK ---
componentes, o dieciocho (18) meses a partir de la entrega a satisfacción de estos por parte de ECOPETROL S.A. Durante este período la garantía debe cubrir la reparación o reemplazo de los componentes defectuosos por parte del VENDEDOR incluyendo todas las labor

In [22]:
# ============================================
# Paso 13.1 – Inventario de sistemas (Skids, TAGs y cantidades)
# ============================================

import json

def prueba_inventario_sistemas():
    pregunta = "¿Cuáles son los skids o sistemas de dosificación requeridos? Dame los TAG y las cantidades."
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 10, "filter": {"type": "table"}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Resultados prueba INVENTARIO (skids + tags + cantidades): {len(results)}")

    kv_count = 0
    for r in results:
        print("\n--- CHUNK INVENTARIO ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

        if "kv" in r.metadata:
            try:
                kv_dict = json.loads(r.metadata["kv"])
                print("KV dict:", kv_dict)
                kv_count += 1
            except Exception:
                print("KV (raw):", r.metadata["kv"])

    print(f"\n📊 Total de chunks con KV útiles: {kv_count}")

# Ejemplo de ejecución
prueba_inventario_sistemas()


🔎 Resultados prueba INVENTARIO (skids + tags + cantidades): 10

--- CHUNK INVENTARIO ---
SKID. -INSTALACIÓN DE CABLEADOS INTERNOS DE ELÉCTRICIDAD E INSTRUMENTACIÓN, ASÍ COMO INTERCONEXIONES DE LOS EQUIPOS ROTATIVOS Y ESTÁTICOS QUE INTEGRAN EL SKID. -SISTEMA DE CONTROL LOCAL DEL SKID, QUE CUENTEN CON PUERTOS DE FIBRA ÓPTICA Y PROTOCOLO MODBUS TCP/IP PARA COMUNICACIÓN CON EL SISTEMA DE CONTROL DEL STAP EC3; CONTROLADOR PCS-009. 2. DE ACUERDO A INFORMACIÓN DEL PROVEEDOR DEL SKID DE QUÍMI
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '54_3', 'page': 54, 'type': 'table', 'kv': '{"caudal_gph": "18", "presion_psig": "100", "hp_motor": "1", "material_tanque": "304SS"}', 'prio': 'alcance'}
KV dict: {'caudal_gph': '18', 'presion_psig': '100', 'hp_motor': '1', 'material_tanque': '304SS'}

--- CHUNK INVENTARIO ---
SKID. -INSTALACIÓN DE CABLEADOS INTERNOS DE ELÉCTRICIDAD E INSTRUMENTACIÓN, ASÍ COMO INTERCONEXIONES DE LOS EQUIPOS ROTATIVOS Y E

In [23]:
# ============================================
# Paso 14 (v3) – Alcance + Datos de diseño de la bomba API 675 (robusto + actualizado)
# ============================================
import re, json
from langchain.schema import Document

def _j(x):
    try:
        return json.loads(x)
    except Exception:
        return {}

def _parse_nums(text: str):
    """Extrae caudal (GPH), presión (psig) y HP desde texto libre (fallback)."""
    caudal = None
    pres = None
    hp = None

    # Caudal "12 - 18 GPH" | "12–18 Gph" | "12 a 18 GPH" | simple "18 GPH"
    m = re.search(r'(\d+(?:[.,]\d+)?)\s*(?:-|–|a|to)\s*(\d+(?:[.,]\d+)?)\s*(gph|g\.?p\.?h\.?)\b', text, re.I)
    if m:
        caudal = f"{m.group(1)} - {m.group(2)}"
    else:
        m = re.search(r'\b(\d+(?:[.,]\d+)?)\s*(gph|g\.?p\.?h\.?)\b', text, re.I)
        if m:
            caudal = m.group(1)

    # Presión "@ 100 psi" | "100 psig"
    m = re.search(r'@?\s*(\d+(?:[.,]\d+)?)\s*(psi|psig)\b', text, re.I)
    if m:
        pres = m.group(1)

    # Potencia "1 HP"
    m = re.search(r'\b(\d+(?:[.,]\d+)?)\s*hp\b', text, re.I)
    if m:
        hp = m.group(1)

    return caudal, pres, hp

def prueba_alcance_y_datos_bomba_v3():
    q_alc = "alcance del suministro paquete de inyección de químicos"
    q_hd  = "bomba dosificadora API 675 caudal presión HP materiales GPH psig"

    # 1) Recuperar alcance
    retr_alc = vectorstore.as_retriever(
        search_kwargs={"k": 12, "filter": {"prio": "alcance"}},
        search_type="mmr"
    )
    alc_docs = retr_alc.get_relevant_documents(q_alc)

    if not alc_docs:  # fallback
        retr_alc_fb = vectorstore.as_retriever(search_kwargs={"k": 12}, search_type="mmr")
        alc_docs = retr_alc_fb.get_relevant_documents("alcance del suministro")

    # 2) Recuperar hojas de datos
    retr_hd = vectorstore.as_retriever(
        search_kwargs={"k": 20, "filter": {"prio": "hoja_datos"}},
        search_type="mmr"
    )
    hd_docs = retr_hd.get_relevant_documents(q_hd)

    # 3) Refuerzo directo desde la colección (KV + texto clave)
    coll = vectorstore._collection
    meta_hd = coll.get(where={"prio": "hoja_datos"}, include=["metadatas", "documents"])
    for md, doc in zip(meta_hd["metadatas"], meta_hd["documents"]):
        if not isinstance(md, dict):
            continue
        if ("kv" in md and any(k in md["kv"] for k in ["caudal_gph", "presion_psig", "hp_motor", "material_tanque", "material_mojado", "bombas_operativas", "bombas_respaldo"])) \
           or ("API 675" in (doc or "") or "GPH" in (doc or "") or "psig" in (doc or "")):
            hd_docs.append(Document(page_content=doc, metadata=md))

    # 4) De-duplicar por (page,type,primeros 60 chars)
    def _key(d): return (d.metadata.get("page"), d.metadata.get("type"), (d.page_content or "")[:60])
    seen = set()
    alc_unique, hd_unique = [], []
    for d in alc_docs:
        k = _key(d)
        if k not in seen:
            seen.add(k); alc_unique.append(d)
    for d in hd_docs:
        k = _key(d)
        if k not in seen:
            seen.add(k); hd_unique.append(d)

    # 5) Extraer KV + regex → síntesis
    datos = {
        "caudal_gph": None,
        "presion_psig": None,
        "hp_motor": None,
        "materiales": set(),
        "api_675": False,
        "bombas_operativas": None,
        "bombas_respaldo": None,
        "paginas_fuente": set(),
    }

    for d in hd_unique:
        md = d.metadata or {}
        if "page" in md:
            datos["paginas_fuente"].add(md["page"])

        # KV directo
        if "kv" in md:
            kv = _j(md["kv"])
            if not datos["caudal_gph"] and kv.get("caudal_gph"):
                datos["caudal_gph"] = kv["caudal_gph"]
            if not datos["presion_psig"] and kv.get("presion_psig"):
                datos["presion_psig"] = kv["presion_psig"]
            if not datos["hp_motor"] and kv.get("hp_motor"):
                datos["hp_motor"] = kv["hp_motor"]
            if kv.get("material_tanque"):
                datos["materiales"].add(kv["material_tanque"])
            if kv.get("material_mojado"):
                datos["materiales"].add(kv["material_mojado"])
            if str(kv.get("api_675")).lower() == "true":
                datos["api_675"] = True
            if kv.get("bombas_operativas"):
                datos["bombas_operativas"] = kv["bombas_operativas"]
            if kv.get("bombas_respaldo"):
                datos["bombas_respaldo"] = kv["bombas_respaldo"]

        # Fallback regex
        if any(v is None for v in (datos["caudal_gph"], datos["presion_psig"], datos["hp_motor"])):
            c, p, h = _parse_nums(d.page_content or "")
            if c and not datos["caudal_gph"]:
                datos["caudal_gph"] = c
            if p and not datos["presion_psig"]:
                datos["presion_psig"] = p
            if h and not datos["hp_motor"]:
                datos["hp_motor"] = h

    # 6) Mostrar resultados
    print(f"🔎 ALCANCE (k={len(alc_unique)}):")
    for r in alc_unique[:5]:
        print(f"\n— p.{r.metadata.get('page')} | {r.metadata.get('type')}")
        print((r.page_content or "")[:350].replace("\n", " "))

    print(f"\n🔎 HOJA DE DATOS (k={len(hd_unique)}):")
    for r in hd_unique[:6]:
        print(f"\n— p.{r.metadata.get('page')} | {r.metadata.get('type')}")
        print((r.page_content or "")[:350].replace("\n", " "))
        if "kv" in r.metadata:
            print("KV:", r.metadata["kv"])

    print("\n✅ EXTRACCIÓN SINTETIZADA:")
    print(f"  • Caudal (GPH): {datos['caudal_gph']}")
    print(f"  • Presión (psig): {datos['presion_psig']}")
    print(f"  • Potencia (HP): {datos['hp_motor']}")
    print(f"  • Materiales: {', '.join(sorted(datos['materiales'])) or '—'}")
    print(f"  • API 675: {'Sí' if datos['api_675'] else 'N/D'}")
    print(f"  • Bombas operativas: {datos['bombas_operativas'] or '—'}")
    print(f"  • Bombas respaldo: {datos['bombas_respaldo'] or '—'}")
    print(f"  • Páginas fuente (HD): {sorted(datos['paginas_fuente'])}")

    return {"alcance": alc_unique, "hoja_datos": hd_unique, "datos": datos}

# Ejecución:
prueba_alcance_y_datos_bomba_v3()



🔎 ALCANCE (k=8):

— p.1 | text
MAYORES PAQUETES DE INYECCIÓN DE QUÍMICOS

— p.18 | text
MAYORES PAQUETES DE INYECCIÓN DE QUÍMICOS

— p.25 | text
MAYORES PAQUETES DE INYECCIÓN DE QUÍMICOS

— p.26 | text
MAYORES PAQUETES DE INYECCIÓN DE QUÍMICOS

— p.2 | text
MAYORES PAQUETES DE INYECCIÓN DE QUÍMICOS

🔎 HOJA DE DATOS (k=106):

— p.43 | text
96 2 agua, para la introducción del polímero sólido, agitacion, alarma para vaciado y llenado. para la dosificacion debe contar con Una (1) bomba dosificadora de diafragma para la inyección de polímero líquido preparado de 1Hp,12 - 18 gph @ 100psi, API 675, con una (1) de respaldo accionadas por motores eléctricos. El paquete contara con un sistema
KV: {"caudal_gph": "18", "presion_psig": "100", "hp_motor": "1", "material_tanque": "304SS", "api_675": "true"}

— p.42 | table
(GPH) | | MODELO | RANGO (Cps) | POTENCIA MOTOR ELECTRICO (hp) (NOTA 15) | CANTIDAD 20 21 22 23 24 25 26 27 28 | STAP EC3 (NOTA 18) AFC4 | Se requiere una unidad paquete de Prepara

{'alcance': [Document(metadata={'page': 1, 'prio': 'alcance', 'chunk_id': '1_1', 'type': 'text', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}, page_content='MAYORES\nPAQUETES DE INYECCIÓN DE QUÍMICOS'),
  Document(metadata={'page': 18, 'type': 'text', 'chunk_id': '18_1', 'prio': 'alcance', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}, page_content='MAYORES\nPAQUETES DE INYECCIÓN DE QUÍMICOS'),
  Document(metadata={'type': 'text', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'alcance', 'page': 25, 'chunk_id': '25_1'}, page_content='MAYORES\nPAQUETES DE INYECCIÓN DE QUÍMICOS'),
  Document(metadata={'type': 'text', 'page': 26, 'prio': 'alcance', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '26_1'}, page_content='MAYORES\nPAQUETES DE INYECCIÓN DE QUÍMICOS'),
  Document(metadata={'prio': 'alcance', 'source': './data/CAS09991MERMR000003_I